## Theoretical Overview
$$ H(x) = Wx + b $$
$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$
$H(x)$: 주어진 $x$ 값에 대해 예측을 어떻게 할 것인가
$cost(W, b)$: $H(x)$ 가 $y$ 를 얼마나 잘 예측했는가

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Pytorch Random Seed
- 딥러닝은 weight initialization 등에 random number 쓸 일이 많다.
- 실험 결과를 재구현하거나 개선이 되었는 지를 비교해보려면 실험을 다시 할 때도 동일한 random number를 사용해야 할 때가 있다.
- random number를 CPU에서 생성하기도 하고 GPU(cuda)에서 생성하기도 한다. 띠리사 들 디 seed를 별도로 관리해야 한다.
- 이번 실험에 사용된 random seed 출력해보려면: torch.initial_seed() or torch.cuda.initial_seed()
- 다음 실험에서 같은 random seed를 사용하고 싶다면: torch.manual_seed(number) or torch.cuda.manual_seed_all(number)

In [2]:
# For reproducibility
# random seed 설정 --> 같은 순서로 난수 발생
torch.manual_seed(1)

## Data
- 데이터는 torch.tensor!
- 입력 따로, 출력 따로

In [3]:
x_train=torch.FloatTensor([[1],[2],[3]])
y_train=torch.FloatTensor([[1],[2],[3]])

In [4]:
print(x_train)
print(x_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


In [5]:
print(y_train)
print(y_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])


## Weight Initialization
- Weight와 Bias 0으로 초기화
- requires_grad=True: 학습할 것이라고 명시(W,b를 학습시킬 것이다.)
### Torch.zeros
torch.zeros(*size, *, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False) → Tensor

### .requires_grad
- True로 설정 시: 그 tensor에서 이뤄지는 모든 연산들을 추적, 학습 가능한 매개변수
- 계산 완료 후, .backward() 호출하여 모든 변화도(gradient) 자동으로 계산


In [6]:
W=torch.zeros(1,requires_grad=True)
print(W)

tensor([0.], requires_grad=True)


In [7]:
b=torch.zeros(1,requires_grad=True)
print(b)

tensor([0.], requires_grad=True)


## Hypothesis
$$ H(x) = Wx + b $$

In [8]:
hypothesis = x_train * W + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


## Cost
$$ cost(W, b) = \frac{1}{m} \sum^m_{i=1} \left( H(x^{(i)}) - y^{(i)} \right)^2 $$

In [9]:
print(hypothesis-y_train)

tensor([[-1.],
        [-2.],
        [-3.]], grad_fn=<SubBackward0>)


In [10]:
print((hypothesis-y_train)**2)

tensor([[1.],
        [4.],
        [9.]], grad_fn=<PowBackward0>)


In [11]:
cost=torch.mean((hypothesis-y_train)**2) # torch.mean: 평균으로 계산
print(cost)

tensor(4.6667, grad_fn=<MeanBackward0>)


## Gradient Descent
1. torch.optim 라이브러리 사용
- [W,b]는 학습할 tensor들
- lr=0.01은 learning rate
2. 항상 붙어다니는 3줄
- optimizer.zero_grad(): gradient 0으로 초기화
- cost.backward(): gradient 계산
- optimizer.step()으로 개선

In [12]:
optimizer=optim.SGD([W,b],lr=0.01)

In [13]:
optimizer.zero_grad()
cost.backward()
optimizer.step()

In [14]:
print(W)
print(b)

tensor([0.0933], requires_grad=True)
tensor([0.0400], requires_grad=True)


In [15]:
# Let's check if the hypothesis is now better.
hypothesis=x_train*W+b
print(hypothesis)

cost=torch.mean((hypothesis-y_train)**2)
print(cost)

tensor([[0.1333],
        [0.2267],
        [0.3200]], grad_fn=<AddBackward0>)
tensor(3.6927, grad_fn=<MeanBackward0>)


## Training with Full Code
현실적으로, multiple epochs로 dataset을 훈련시킨다. 이는 반복문으로 간단하게 구현할 수 있다.

In [16]:
# 데이터
x_train=torch.FloatTensor([[1],[2],[3]])
y_train=torch.FloatTensor([[1],[2],[3]])
# 모델 초기화
W=torch.zeros(1,requires_grad=True)
b=torch.zeros(1, requires_grad=True)
# Optimizer 설정
optimizer=optim.SGD([W,b],lr=0.01)

nb_epochs=1000
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    hypothesis=x_train*W+b
    
    # cost 계산
    cost=torch.mean((hypothesis-y_train)**2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch%100==0:
        print('Epoch {:4d}/{} W: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))


Epoch    0/1000 W: 0.093 Cost: 0.040000
Epoch  100/1000 W: 0.873 Cost: 0.289036
Epoch  200/1000 W: 0.900 Cost: 0.227211
Epoch  300/1000 W: 0.921 Cost: 0.178608
Epoch  400/1000 W: 0.938 Cost: 0.140402
Epoch  500/1000 W: 0.951 Cost: 0.110369
Epoch  600/1000 W: 0.962 Cost: 0.086760
Epoch  700/1000 W: 0.970 Cost: 0.068201
Epoch  800/1000 W: 0.976 Cost: 0.053612
Epoch  900/1000 W: 0.981 Cost: 0.042144
Epoch 1000/1000 W: 0.985 Cost: 0.033129


### tensor.item()
- tensor에 하나의 값만 존재한다면 이를 사용해서 숫자 값(스칼라)을 얻을 수 있다.

In [17]:
print(W)
print(W.item())

tensor([0.9854], requires_grad=True)
0.9854263663291931


## High-level Implementation with nn.Module
Linear regression 모델을 만들 때, 기본적으로 PyTorch의 모든 모델은 제공되는 nn.Module을 inherit해서 만든다.
- nn.Module을 상속해서 모델 생성
- nn.Linear(3,1)
1. 입력 차원: 3
2. 출력 차원: 1

- Hypothesis(predict) 계산은 forward()에서!
- Gradient 계산은 PyTorch가 알아서 해준다

-----------

모델의 __ init __ 에서는 사용할 레이어들을 정의하게 된다.  
여기서 우리는 linear regression 모델을 만들기 때문에, nn.Linear을 이용한다. 그리고 forward 에서는 이 모델이 어떻게 입력값에서 출력값을 계산하는지 알려준다.

In [18]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=nn.Linear(1,1)
        
    def forward(self,x):
        return self.linear(x)

In [19]:
model=LinearRegressionModel()

## Hypothesis

이제 모델을 생성해서 예측값 H(x)를 구한다.

In [20]:
hypothesis=model(x_train)

In [21]:
print(hypothesis)

tensor([[0.0739],
        [0.5891],
        [1.1044]], grad_fn=<AddmmBackward>)


## Cost
### F.mse_loss
- torch.nn.Functional 에서 제공하는 loss function 사용
- 쉽게 다른 loss와 교체 가능

-----
이제 mean squared error(MSE)로 cost를 구한다. MSE 역시 PyTorch에서 기본적으로 제공한다.

In [22]:
print(hypothesis)
print(y_train)

tensor([[0.0739],
        [0.5891],
        [1.1044]], grad_fn=<AddmmBackward>)
tensor([[1.],
        [2.],
        [3.]])


In [23]:
cost=F.mse_loss(hypothesis, y_train)

In [24]:
print(cost)

tensor(2.1471, grad_fn=<MseLossBackward>)


## Gradient Descent
마지막 주어진 cost를 이용해 H(x)의 W,b를 바꾸어서 cost를 줄여본다. 이때, PyTorch의 torch.optim에 있는 optimizer들 중 하나를 사용할 수 있다.

In [25]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [26]:
optimizer.zero_grad()
cost.backward()
optimizer.step()

## Training with Full Code

In [27]:
# 데이터
x_train=torch.FloatTensor([[1],[2],[3]])
y_train=torch.FloatTensor([[1],[2],[3]])
# 모델 초기화
model=LinearRegressionModel()
# optimizer 설정
optimizer=optim.SGD(model.parameters(), lr=0.01)

nb_epochs=1000
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # Cost 계산
    cost=F.mse_loss(prediction,y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch%100==0:
        params=list(model.parameters())
        W=params[0].item()
        b=params[1].item()
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W, b, cost.item()
        ))

Epoch    0/1000 W: -0.101, b: 0.508 Cost: 4.630286
Epoch  100/1000 W: 0.713, b: 0.653 Cost: 0.061555
Epoch  200/1000 W: 0.774, b: 0.514 Cost: 0.038037
Epoch  300/1000 W: 0.822, b: 0.404 Cost: 0.023505
Epoch  400/1000 W: 0.860, b: 0.317 Cost: 0.014525
Epoch  500/1000 W: 0.890, b: 0.250 Cost: 0.008975
Epoch  600/1000 W: 0.914, b: 0.196 Cost: 0.005546
Epoch  700/1000 W: 0.932, b: 0.154 Cost: 0.003427
Epoch  800/1000 W: 0.947, b: 0.121 Cost: 0.002118
Epoch  900/1000 W: 0.958, b: 0.095 Cost: 0.001309
Epoch 1000/1000 W: 0.967, b: 0.075 Cost: 0.000809
